## Package management in Python
#### Background information
* The standard package manager in python is pip (Pip Installs Packages)
* The standard central repository for python packages is PyPI (Python Package Index)
* For comparison in the future, here's my system's default python installation on my personal laptop.  It has version 2.7.16 and doesn't include pip.

![My system's default python installation](no-environment.png "System Base Python")

#### What if I want to use different version of Python?  Like python 2.8 or 3?

Well, you could install another version of python and add it to your path.

Then what happens when you use pip?  You can't really be sure without additional information.  You'd have to know which version of python is first on your path, and maybe specify the whole path to the python you want to use. to be sure.

#### What if I have two projects that depend on conflicting versions of the same package?
Python can't handle this on its own.  If ProjectA requires basicpackage 2.0 and ProjectB requires basicpackage 3.0, you're out of luck.

I used to waste a lot of time installing packages, realizing I'd installed them to the wrong Python, and reinstalling them.  Or thinking my code was broken because I ran `python my_script.py` instead of `python3`.

*Until...*

## Enter the Virtual Environment!
#### What is a virtual environment?
* A virtual environment is a way to easily create a fresh python installation for your project.  Each one can have its own packages with whatever versions it needs.


#### Why should I use one?
* It resolves the problems with conflicting Python versions and conflicting package versions that I described above.
* It is Best Practice™ to have a separate virtual environment for every project you work on.
* You can put a virtual environment in a directory you control. This means that at work, I never have to worry about the fact that I don't have sudo privileges for the Python installations in /opt/.

#### How does it work?
* A virtual environment creates a new Python installation directory.
* In the bin/ subdirectory, it has hardlinks to the binaries of another, "base" Python installation directory.
* *However*, it points to its own new directories where packages can be installed.
* When we activate a virtual environment, its python becomes the main python on your terminal.
* When we deactivate it, everything goes back to how it was before.

There are several packages that allow for creating python virtual environments (venv and pyenv among them) but my favorite is...
## Conda
* Conda is an open source package *and* environment management system that runs on Windows, macOS and Linux.
* Conda easily creates, saves, loads and switches between environments on your local computer.
* It was created for Python programs (and that's where it is most popular), but it can theoretically package and distribute software for any language (whereas pip and PyPI are only for python).
* Anaconda, Inc. maintains its own curated repository of 1000+ packages for data science.  Conda can install these with `conda install`.
* If you want something they don't offer, you can still use pip from within a conda environment.

![Switching from Conda's base environment, to my default environment, to the environment I created to make this presentation](switching-environments.png "Switching Between Conda Environments")

No more worrying about which python installation to use:  Just run `python my_script.py` and the python of your current environment will be used to run it.

#### What's the difference between Miniconda and Anaconda?
* Anaconda and Miniconda are both distributions.  They come with python, conda, and some packages.
* Miniconda comes with a minimal python installation and the conda package and environment manager.
* Anaconda comes with an entire PyData ecosystem of over 1000 libraries by default, including numpy, scipy, pandas, sklearn, and tensorflow.
* Anaconda requires several GB of disk space and since I haven't done much Python data science at BBN, I have miniconda and just install things as I need them.
* If you have one distribution, for example miniconda with python 2.7, you can add other versions of python with `conda install python=3.6`.  Then when you create new environments, you can specify which python version to use with `conda create -n my_env_name python=3.6`.

#### Is pyinstaller compatible with virtual environments?
* Yes.  Activate your environment, install pyinstaller, and run pyinstaller from within the environment.